# Self-Driving Car Engineer Nanodegree


## Project: Build a Traffic Light Detector for CarND-Capstone project

In this notebook, we are building a Traffic Light Detector for the Capstone project for the Udacity Self Driving Cars nanodegree. 

In addition to implementing code, there is a [write up file](https://github.com/edupaz2/Traffic-Light-Classifier-Project/blob/master/writeup.md).

## Step 0: Prepare the Data

In [33]:
# Transform JPG to PNG

def jpg2png(jpgs_list):
    for j in jpgs_list:
        # Convert to PNG
        img = cv2.imread(j)
        print(j[:-3] + 'png')
        cv2.imwrite(j[:-3] + 'png', img)
        # Delete the file
        os.remove(j)

transformJPG2PNG = False
if transformJPG2PNG:
    l = glob.glob('./sim-data/**/*.jpg', recursive=True)
    jpg2png(l)


print("Done")


Done


In [34]:
# Rename files Consecutively

def renameConsecutively(path):
    counter = 1
    filelist = os.listdir(path)
    zerofill = len(str(abs(len(filelist))))
    for f in filelist:
        oldname = '{0}/{1}'.format(path, f)
        newname = '{0}/{1}.png'.format(path, str(counter).zfill(zerofill))
        os.rename(oldname, newname)
        counter += 1

rename = False
if rename:
    renameConsecutively('sim-data/Green')
    renameConsecutively('sim-data/None')
    renameConsecutively('sim-data/Red')
    renameConsecutively('sim-data/Yellow')

print('Done')

Done
